In [1]:
import pandas as pd
import string
import re
from functools import reduce
import math
from typing import Union

/Users/issacj/Desktop/Synapxe/healthhub-content-optimization/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext kedro.ipython

[07/20/24 21:59:48] INFO     Registered line magic '%reload_kedro'                                   ]8;id=676861;file:///Users/issacj/Desktop/Synapxe/healthhub-content-optimization/env/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=867122;file:///Users/issacj/Desktop/Synapxe/healthhub-content-optimization/env/lib/python3.11/site-packages/kedro/ipython/__init__.py#58\58]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=762305;file:///Users/issacj/Desktop/Synapxe/healthhub-content-optimization/env/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=34402;file:///Users/issacj/Desktop/Synapxe/healthhub-content-optimization/env/lib/python3.11/site-packages/kedro/ipython/__init__.py#60\60]8;;\

                    WARNING  Kedro extension was registered but couldn't find a Kedro project. Make  ]8;id=876361;file:///Users/issacj/Desktop/Synapxe/healthhub-content-optimization/env/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=486393;file:///Users/issacj/Desktop/Synapxe/healthhub-content-optimization/env/lib/python3.11/site-packages/kedro/ipython/__init__.py#63\63]8;;\
                             sure you run '%reload_kedro <project_root>'.                                          

In [3]:
merged_data = pd.read_parquet("../data/merged_data.parquet")

display(merged_data)

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,to_remove,remove_type,has_table,has_image,related_sections,extracted_tables,extracted_links,extracted_headers,extracted_img_alt_text,extracted_content_body
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/breas...,breast-cancer-screening-subsidies,Here’s all you need to know about breast cance...,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",...,False,None,True,False,[Cancer Facts You Cannot Ignore],[[[National Healthcare Group (NHG) Polyclinics...,"[[Cancer Facts You Cannot Ignore, https://www....","[[Breast Cancer Screening, h2], [Subsidy for M...",[],Breast cancer is the number one cancer among w...
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/marri...,marriage_parenthood_scheme,New parents and couples looking to conceive ca...,"b'<div class=""ExternalClassE1D82270F17241E4955...",...,False,None,True,False,"[MediSave, Baby Bonus - What You Need to Know,...","[[[Delivery Procedure, No. of Days of Hospital...","[[How to Submit Claims, https://crms.moh.gov.s...","[[MediSave Maternity Package, h2], [Examples o...",[],MediSave Maternity Package\nWith the MediSave ...
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/medisave,medisave,MediSave is the national medical savings schem...,"b'<div class=""ExternalClass67AD25F1F8B64B349E5...",...,False,None,False,False,[A Way to Reduce Outpatient Cost for Chronic D...,None,[[A Way to Reduce Outpatient Cost for Chronics...,"[[, h2], [What is MediSave?, h2], [Contributin...",[],"What is MediSave?\nMediSave, introduced in Apr..."
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/finan...,financial-assistance-for-local-patients-in-sin...,Having trouble paying your medical bill? Here’...,"b'<div class=""ExternalClassE335708125E743FDAA3...",...,False,None,False,False,[],None,"[[Medifund, https://www.healthhub.sg/a-z/costs...","[[Medifund: Hospital Bill Payment Assistance, ...",[],Patients or family members who have difficulty...
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/chas,chas,"With a CHAS card, all Singapore citizens can r...",b'<h2>What is the Community Health Assist Sche...,...,False,None,True,True,[All Pioneer Generation (PG) and Merdeka Gener...,"[[[CHAS Criteria, CHAS Green, CHAS Orange, CHA...","[[www.chas.sg/apply, https://www.chas.sg/apply...",[[What is the Community Health Assist Scheme (...,"[chas green card, chas blue card, CHAS_Subsidy...",What is the Community Health Assist Scheme (CH...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2608,1440763,Heart Failure Transitional Care Programme,Heart Failure Transitional Care Programme,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/support-groups-an...,www.healthhub.sg/a-z/support-groups-and-others...,transitional-care-programme-for-heart-failure,The team from NUHCS gives support to heart fai...,"b'<div class=""ExternalClassFC126593610D4F05

In [4]:
df_keep = merged_data[~merged_data["to_remove"]]

display(df_keep)

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,to_remove,remove_type,has_table,has_image,related_sections,extracted_tables,extracted_links,extracted_headers,extracted_img_alt_text,extracted_content_body
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/breas...,breast-cancer-screening-subsidies,Here’s all you need to know about breast cance...,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",...,False,None,True,False,[Cancer Facts You Cannot Ignore],[[[National Healthcare Group (NHG) Polyclinics...,"[[Cancer Facts You Cannot Ignore, https://www....","[[Breast Cancer Screening, h2], [Subsidy for M...",[],Breast cancer is the number one cancer among w...
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/marri...,marriage_parenthood_scheme,New parents and couples looking to conceive ca...,"b'<div class=""ExternalClassE1D82270F17241E4955...",...,False,None,True,False,"[MediSave, Baby Bonus - What You Need to Know,...","[[[Delivery Procedure, No. of Days of Hospital...","[[How to Submit Claims, https://crms.moh.gov.s...","[[MediSave Maternity Package, h2], [Examples o...",[],MediSave Maternity Package\nWith the MediSave ...
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/medisave,medisave,MediSave is the national medical savings schem...,"b'<div class=""ExternalClass67AD25F1F8B64B349E5...",...,False,None,False,False,[A Way to Reduce Outpatient Cost for Chronic D...,None,[[A Way to Reduce Outpatient Cost for Chronics...,"[[, h2], [What is MediSave?, h2], [Contributin...",[],"What is MediSave?\nMediSave, introduced in Apr..."
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/finan...,financial-assistance-for-local-patients-in-sin...,Having trouble paying your medical bill? Here’...,"b'<div class=""ExternalClassE335708125E743FDAA3...",...,False,None,False,False,[],None,"[[Medifund, https://www.healthhub.sg/a-z/costs...","[[Medifund: Hospital Bill Payment Assistance, ...",[],Patients or family members who have difficulty...
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/chas,chas,"With a CHAS card, all Singapore citizens can r...",b'<h2>What is the Community Health Assist Sche...,...,False,None,True,True,[All Pioneer Generation (PG) and Merdeka Gener...,"[[[CHAS Criteria, CHAS Green, CHAS Orange, CHA...","[[www.chas.sg/apply, https://www.chas.sg/apply...",[[What is the Community Health Assist Scheme (...,"[chas green card, chas blue card, CHAS_Subsidy...",What is the Community Health Assist Scheme (CH...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1440774,Breast Cancer Support Groups,Breast Cancer Support Groups,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/support-groups-an...,www.healthhub.sg/a-z/support-groups-and-others...,Breast-cancer-support-group-singapore,Learn how breast cancer support groups help br...,"b'<div class=""ExternalClassE9019B25CD2A40F3948...",...,False,None,False,False

In [5]:
relevant_categories = [
    "cost-and-financing",
    "live-healthy-articles",
    "diseases-and-conditions",
    "medical-care-and-facilities",
    "support-group-and-others",
]

df_keep = df_keep[df_keep["content_category"].isin(relevant_categories)]

display(df_keep)

,id,content_name,title,article_category_names,cover_image_url,full_url,full_url2,friendly_url,category_description,content_body,...,to_remove,remove_type,has_table,has_image,related_sections,extracted_tables,extracted_links,extracted_headers,extracted_img_alt_text,extracted_content_body
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/breas...,breast-cancer-screening-subsidies,Here’s all you need to know about breast cance...,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",...,False,None,True,False,[Cancer Facts You Cannot Ignore],[[[National Healthcare Group (NHG) Polyclinics...,"[[Cancer Facts You Cannot Ignore, https://www....","[[Breast Cancer Screening, h2], [Subsidy for M...",[],Breast cancer is the number one cancer among w...
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/marri...,marriage_parenthood_scheme,New parents and couples looking to conceive ca...,"b'<div class=""ExternalClassE1D82270F17241E4955...",...,False,None,True,False,"[MediSave, Baby Bonus - What You Need to Know,...","[[[Delivery Procedure, No. of Days of Hospital...","[[How to Submit Claims, https://crms.moh.gov.s...","[[MediSave Maternity Package, h2], [Examples o...",[],MediSave Maternity Package\nWith the MediSave ...
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/medisave,medisave,MediSave is the national medical savings schem...,"b'<div class=""ExternalClass67AD25F1F8B64B349E5...",...,False,None,False,False,[A Way to Reduce Outpatient Cost for Chronic D...,None,[[A Way to Reduce Outpatient Cost for Chronics...,"[[, h2], [What is MediSave?, h2], [Contributin...",[],"What is MediSave?\nMediSave, introduced in Apr..."
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/finan...,financial-assistance-for-local-patients-in-sin...,Having trouble paying your medical bill? Here’...,"b'<div class=""ExternalClassE335708125E743FDAA3...",...,False,None,False,False,[],None,"[[Medifund, https://www.healthhub.sg/a-z/costs...","[[Medifund: Hospital Bill Payment Assistance, ...",[],Patients or family members who have difficulty...
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/costs-and-financi...,www.healthhub.sg/a-z/costs-and-financing/chas,chas,"With a CHAS card, all Singapore citizens can r...",b'<h2>What is the Community Health Assist Sche...,...,False,None,True,True,[All Pioneer Generation (PG) and Merdeka Gener...,"[[[CHAS Criteria, CHAS Green, CHAS Orange, CHA...","[[www.chas.sg/apply, https://www.chas.sg/apply...",[[What is the Community Health Assist Scheme (...,"[chas green card, chas blue card, CHAS_Subsidy...",What is the Community Health Assist Scheme (CH...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1440774,Breast Cancer Support Groups,Breast Cancer Support Groups,"Conditions and Illnesses,",https://ch-api.healthhub.sg/api/public/content...,https://www.healthhub.sg/a-z/support-groups-an...,www.healthhub.sg/a-z/support-groups-and-others...,Breast-cancer-support-group-singapore,Learn how breast cancer support groups help br...,"b'<div class=""ExternalClassE9019B25CD2A40F3948...",...,False,None,False,False

In [6]:
df_extracted = df_keep[
    [
        "id",
        "content_name",
        "title",
        "article_category_names",
        "full_url",
        "friendly_url",
        "category_description",
        "content_category",
        "content_body",
        "pr_name",
        "has_table",
        "has_image",
        "related_sections",
        "extracted_tables",
        "extracted_links",
        "extracted_headers",
        "extracted_img_alt_text",
        "extracted_content_body",
    ]
]

display(df_extracted)

,id,content_name,title,article_category_names,full_url,category_description,content_category,content_body,pr_name,has_table,has_image,related_sections,extracted_tables,extracted_links,extracted_headers,extracted_img_alt_text,extracted_content_body
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/costs-and-financi...,Here’s all you need to know about breast cance...,cost-and-financing,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",Health Promotion Board,True,False,[Cancer Facts You Cannot Ignore],[[[National Healthcare Group (NHG) Polyclinics...,"[[Cancer Facts You Cannot Ignore, https://www....","[[Breast Cancer Screening, h2], [Subsidy for M...",[],Breast cancer is the number one cancer among w...
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://www.healthhub.sg/a-z/costs-and-financi...,New parents and couples looking to conceive ca...,cost-and-financing,"b'<div class=""ExternalClassE1D82270F17241E4955...",Ministry of Health,True,False,"[MediSave, Baby Bonus - What You Need to Know,...","[[[Delivery Procedure, No. of Days of Hospital...","[[How to Submit Claims, https://crms.moh.gov.s...","[[MediSave Maternity Package, h2], [Examples o...",[],MediSave Maternity Package\nWith the MediSave ...
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://www.healthhub.sg/a-z/costs-and-financi...,MediSave is the national medical savings schem...,cost-and-financing,"b'<div class=""ExternalClass67AD25F1F8B64B349E5...",Ministry of Health,False,False,[A Way to Reduce Outpatient Cost for Chronic D...,None,[[A Way to Reduce Outpatient Cost for Chronics...,"[[, h2], [What is MediSave?, h2], [Contributin...",[],"What is MediSave?\nMediSave, introduced in Apr..."
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://www.healthhub.sg/a-z/costs-and-financi...,Having trouble paying your medical bill? Here’...,cost-and-financing,"b'<div class=""ExternalClassE335708125E743FDAA3...",Khoo Teck Puat Hospital,False,False,[],None,"[[Medifund, https://www.healthhub.sg/a-z/costs...","[[Medifund: Hospital Bill Payment Assistance, ...",[],Patients or family members who have difficulty...
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://www.healthhub.sg/a-z/costs-and-financi...,"With a CHAS card, all Singapore citizens can r...",cost-and-financing,b'<h2>What is the Community Health Assist Sche...,CHAS,True,True,[All Pioneer Generation (PG) and Merdeka Gener...,"[[[CHAS Criteria, CHAS Green, CHAS Orange, CHA...","[[www.chas.sg/apply, https://www.chas.sg/apply...",[[What is the Community Health Assist Scheme (...,"[chas green card, chas blue card, CHAS_Subsidy...",What is the Community Health Assist Scheme (CH...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1440774,Breast Cancer Support Groups,Breast Cancer Support Groups,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,Learn how breast cancer support groups help br...,support-group-and-others,"b'<div class=""ExternalClassE9019B25CD2A40F3948...","National University Cancer Institute, Singapore",False,False,[],None,"[[here, https://www.ncis.com.sg/events/Pages/E...","[[Breast Cancer Support at NCIS, h2], [Our Mis...",[],"NCIS Breast Support Group provides knowledge, ..."
2608,1440763,Heart Failure Transitional Care Programme,Heart Failure Transitional Care Programme,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,The team from NUHCS gives support to heart fai...,support-group-and-others,"b'<div class=""ExternalClassFC126593610D4F0587A...","National University Heart Centre, Singapore",False,True,[],None,"[[charles_wu@nuhs.edu.sg, mailto: charles_wu@n...",[[Transitional Care for Heart Failure Patients...,"[Trans

In [7]:
def hemmingway_score(text: str) -> dict[str, Union[str, int]]:
    """
    Calculates the hemmingway score of the given text.

    Args:
        text (str): The text to be scored.

    Returns:
        tuple(int, str): Returns a tuple containing the hemmingway score and difficulty of the text
    """
    sentences = []
    words = []

    # Compile regex to detect punctuations
    regex = re.compile("[%s]" % re.escape(string.punctuation))

    # Remove all hyperlinks
    filtered_text = re.sub(r"https?:\/\/[^\s]+", "", text)

    # Split the extracted text by the newline delimiter
    lines = filtered_text.split("\n")
    # Track the sentences in the text
    for line in lines:
        partial_sentences = re.split(r"[.!?]", line)
        for sentence in partial_sentences:
            sentences.append(sentence.strip())

    # Track the words in the text
    for sentence in sentences:
        sentence_words = sentence.split(" ")
        for word in sentence_words:
            word = regex.sub("", word.strip())
            words.append(word)

    # Filter for empty strings
    filtered_sentences = list(filter(lambda x: len(x) > 0, sentences))
    filtered_words = list(filter(lambda x: len(x) > 0, words))

    # Count the number of sentences, words and letters
    num_sentences = len(filtered_sentences)
    num_words = len(filtered_words)
    num_letters = reduce(lambda x, y: x + y, map(len, words))

    # Calculate the Hemmingway Score
    score = math.ceil(
        4.71 * (num_letters / num_words) + 0.5 * (num_words / num_sentences) - 21.43
    )

    # Get the reading level of the text based on the calculated score
    if score < 10:
        level = "normal"
    elif 10 <= score < 14:
        level = "hard"
    else:
        level = "very hard"

    return {"score": score, "level": level}

In [8]:
def calculate_score(text):
    metrics = hemmingway_score(text)
    return metrics["score"]

def calculate_level(text):
    metrics = hemmingway_score(text)
    return metrics["level"]

df_extracted["readability_score"] = df_extracted["extracted_content_body"].apply(calculate_score)
df_extracted["reading_level"] = df_extracted["extracted_content_body"].apply(calculate_level)
display(df_extracted)

                    WARNING  /var/folders/3n/y5_h0fxs0bv2mhb7bf_fpmb80000gn/T/ipykernel_35144/29377 ]8;id=659632;file:///opt/homebrew/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=505643;file:///opt/homebrew/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py#110\110]8;;\
                             53114.py:9: SettingWithCopyWarning:                                                   
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df_extracted["readability_score"] =                                                 
                             df_extracted["extracted_content_body"].apply(calculate_score)                         
                                                                                                                   

                    WARNING  /var/folders/3n/y5_h0fxs0bv2mhb7bf_fpmb80000gn/T/ipykernel_35144/29377 ]8;id=117453;file:///opt/homebrew/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py\warnings.py]8;;\:]8;id=757430;file:///opt/homebrew/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/warnings.py#110\110]8;;\
                             53114.py:10: SettingWithCopyWarning:                                                  
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc[row_indexer,col_indexer] = value instead                               
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df_extracted["reading_level"] =                                                     
                             df_extracted["extracted_content_body"].apply(calculate_level)                         
                                                                                                                   

,id,content_name,title,article_category_names,full_url,category_description,content_category,content_body,pr_name,has_table,has_image,related_sections,extracted_tables,extracted_links,extracted_headers,extracted_img_alt_text,extracted_content_body,readability_score,reading_level
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/costs-and-financi...,Here’s all you need to know about breast cance...,cost-and-financing,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",Health Promotion Board,True,False,[Cancer Facts You Cannot Ignore],[[[National Healthcare Group (NHG) Polyclinics...,"[[Cancer Facts You Cannot Ignore, https://www....","[[Breast Cancer Screening, h2], [Subsidy for M...",[],Breast cancer is the number one cancer among w...,9,normal
1,1435071,Marriage and Parenthood Schemes,Marriage and Parenthood Schemes,"Body Care,",https://www.healthhub.sg/a-z/costs-and-financi...,New parents and couples looking to conceive ca...,cost-and-financing,"b'<div class=""ExternalClassE1D82270F17241E4955...",Ministry of Health,True,False,"[MediSave, Baby Bonus - What You Need to Know,...","[[[Delivery Procedure, No. of Days of Hospital...","[[How to Submit Claims, https://crms.moh.gov.s...","[[MediSave Maternity Package, h2], [Examples o...",[],MediSave Maternity Package\nWith the MediSave ...,11,hard
2,1434993,MediSave,MediSave,"Alerts & Advisories,",https://www.healthhub.sg/a-z/costs-and-financi...,MediSave is the national medical savings schem...,cost-and-financing,"b'<div class=""ExternalClass67AD25F1F8B64B349E5...",Ministry of Health,False,False,[A Way to Reduce Outpatient Cost for Chronic D...,None,[[A Way to Reduce Outpatient Cost for Chronics...,"[[, h2], [What is MediSave?, h2], [Contributin...",[],"What is MediSave?\nMediSave, introduced in Apr...",10,hard
3,1435031,Hospital Bills Financial Assistance in Singapore,Hospital Bills Financial Assistance in Singapore,"Body Care,",https://www.healthhub.sg/a-z/costs-and-financi...,Having trouble paying your medical bill? Here’...,cost-and-financing,"b'<div class=""ExternalClassE335708125E743FDAA3...",Khoo Teck Puat Hospital,False,False,[],None,"[[Medifund, https://www.healthhub.sg/a-z/costs...","[[Medifund: Hospital Bill Payment Assistance, ...",[],Patients or family members who have difficulty...,14,very hard
4,1435043,Community Health Assist Scheme (CHAS) Singapore,Community Health Assist Scheme (CHAS) Singapore,"Alerts and Advisories,",https://www.healthhub.sg/a-z/costs-and-financi...,"With a CHAS card, all Singapore citizens can r...",cost-and-financing,b'<h2>What is the Community Health Assist Sche...,CHAS,True,True,[All Pioneer Generation (PG) and Merdeka Gener...,"[[[CHAS Criteria, CHAS Green, CHAS Orange, CHA...","[[www.chas.sg/apply, https://www.chas.sg/apply...",[[What is the Community Health Assist Scheme (...,"[chas green card, chas blue card, CHAS_Subsidy...",What is the Community Health Assist Scheme (CH...,9,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1440774,Breast Cancer Support Groups,Breast Cancer Support Groups,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,Learn how breast cancer support groups help br...,support-group-and-others,"b'<div class=""ExternalClassE9019B25CD2A40F3948...","National University Cancer Institute, Singapore",False,False,[],None,"[[here, https://www.ncis.com.sg/events/Pages/E...","[[Breast Cancer Support at NCIS, h2], [Our Mis...",[],"NCIS Breast Support Group provides knowledge, ...",12,hard
2608,1440763,Heart Failure Transitional Care Programme,Heart Failure Transitional Care Programme,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/support-groups-an...,The team from NUHCS gives support to heart fai...,support-group-and-others,"b'<div class=""ExternalClassFC126593610D4F0587A...","National University Heart Centre, Singapore",False,True,[],None,"[[charles_wu@

In [9]:
print(df_extracted.groupby(["content_category"])["id"].count().sort_values(ascending=False))

content_category
live-healthy-articles          1105
diseases-and-conditions         310
medical-care-and-facilities      57
cost-and-financing               23
support-group-and-others         12
Name: id, dtype: int64


In [10]:
print(df_extracted.groupby(["pr_name"])["id"].count().sort_values(ascending=False))

pr_name
Health Promotion Board                             666
KK Women's and Children's Hospital                 260
National Healthcare Group                          108
Agency for Integrated Care                          43
Changi General Hospital                             40
National University Hospital                        36
JurongHealth                                        34
Tan Tock Seng Hospital                              33
SingHealth                                          33
Khoo Teck Puat Hospital                             26
Institute of Mental Health                          18
Stroke Services Improvement Team                    18
Singapore General Hospital                          18
National Skin Centre                                18
Health Sciences Authority                           16
Ministry of Health                                  16
Singapore Dental Health Foundation                  15
Alexandra Health System                             14
Na

In [11]:
print(df_extracted.groupby(["article_category_names"])["id"].count().sort_values(ascending=False))

article_category_names
Conditions and Illnesses,                                                     326
Body Care,                                                                    251
Food and Nutrition,                                                           172
Exercise and Fitness,                                                          94
Mind and Balance,                                                              65
                                                                             ... 
Conditions and Illnesses,Alerts and Advisories,                                 1
Food and Nutrition,Intoxicates and Addictions,Pregnancy and Infant Health,      1
Food and Nutrition,Pregnancy and Infant Health,                                 1
Body Care,Exercise & Fitness,Food & Nutrition,                                  1
Body Care,FIGHT and Travel Health,                                              1
Name: id, Length: 82, dtype: int64


In [12]:
print(df_extracted.groupby(["title"])["id"].count().sort_values(ascending=False))

title
Coronary Artery Disease                                                                 2
"The Talk": What to say to your Teen about Sex and Sexuality                            1
Positive Mental Well-being—The Foundation for a Flourishing Life                        1
Prediabetes                                                                             1
Pre-pregnancy Vaccination                                                               1
                                                                                       ..
Eye Care - Is AI The Future?                                                            1
External Ear Infection (Otitis Externa)                                                 1
Exploring Indian Grocery Stores in Singapore - Finding Fresh and Healthy Ingredients    1
Exploring Delicious and Nutritious Vegetarian Delights from Indian Cuisine              1
“Mornings Are the Best Time to Exercise” and Other Myths                                1
Name

In [13]:
print(df_extracted.groupby(["content_name"])["id"].count().sort_values(ascending=False))

content_name
Coronary Artery Disease                                         2
"The Talk": What to say to your Teen about Sex and Sexuality    1
Pregnant When You Have Diabetes: A Checklist                    1
Pregnancy Weight Gain Checklist                                 1
Pregnancy Symptoms                                              1
                                                               ..
Fad Diets: Are They Effective Weight Loss Diets?                1
Facts on Tobacco Smoking and Tobacco Products                   1
Facts and Myths About Healthy Ageing                            1
FAQs About Antenatal Scans                                      1
“Mornings Are the Best Time to Exercise” and Other Myths        1
Name: id, Length: 1506, dtype: int64


In [16]:
print(df_extracted.groupby(["reading_level"])["id"].count().sort_values(ascending=False))

reading_level
normal       1186
hard          295
very hard      26
Name: id, dtype: int64


In [14]:
df_hpb = df_extracted[df_extracted["pr_name"] == "Health Promotion Board"]

display(df_hpb)

,id,content_name,title,article_category_names,full_url,category_description,content_category,content_body,pr_name,has_table,has_image,related_sections,extracted_tables,extracted_links,extracted_headers,extracted_img_alt_text,extracted_content_body,readability_score,reading_level
0,1435040,Breast Screening Subsidies in Singapore,Breast Screening Subsidies in Singapore,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/costs-and-financi...,Here’s all you need to know about breast cance...,cost-and-financing,"b'<div class=""ExternalClass07C58E0D957B4AA7B14...",Health Promotion Board,True,False,[Cancer Facts You Cannot Ignore],[[[National Healthcare Group (NHG) Polyclinics...,"[[Cancer Facts You Cannot Ignore, https://www....","[[Breast Cancer Screening, h2], [Subsidy for M...",[],Breast cancer is the number one cancer among w...,9,normal
28,1437742,Preventing HPV infection: HPV Vaccination,Preventing HPV infection: HPV Vaccination,"Sexual Health and Relationships,Child and Teen...",https://www.healthhub.sg/a-z/diseases-and-cond...,The Human Papillomavirus (HPV) vaccine or HPV ...,diseases-and-conditions,"b'<div class=""ExternalClass50E49E9021B24103B87...",Health Promotion Board,True,True,[],"[[[Cervarix (HPV2 vaccine), Gardasil 9 (HPV9 v...","[[screened, /a-z/costs-and-financing/cervical-...","[[About HPV and its link to Cervical Cancer, h...",[],About HPV and its link to Cervical Cancer\n\nW...,10,hard
29,1437732,Fever in Children,Fever in Children,"Conditions and Illnesses,",https://www.healthhub.sg/a-z/diseases-and-cond...,Spot the symptoms of fever in your child and l...,diseases-and-conditions,"b'<div class=""ExternalClassA0AA5FCB88FC42E4AEF...",Health Promotion Board,False,False,"[Give Your Child the Best Protection, Fever]",None,"[[Upper Respiratory Tract Infections, https://...","[[What is a Fever in a Child?, h2], [How to Me...",[],"What is a Fever in a Child?\nA normal, healthy...",6,normal
30,1437433,"Hand, Foot, and Mouth Disease","Hand, Foot, and Mouth Disease",None,https://www.healthhub.sg/a-z/diseases-and-cond...,"Hand, Foot, and Mouth Disease is present all y...",diseases-and-conditions,"b'<div class=""ExternalClassB1300026922048DE830...",Health Promotion Board,False,True,"[Hand, Foot, and Mouth Disease, Skin Disorders...",None,"[[Hand, Foot and Mouth Disease, https://www.he...","[[Signs and Symptoms, h2], [Complications, h2]...","[Symptoms of Hand, Food and Mouth Disease]","Hand, Foot, and Mouth Disease (HFMD) is caused...",7,normal
32,1437894,Cervical Cancer,Cervical Cancer,"Body Care,",https://www.healthhub.sg/a-z/diseases-and-cond...,Cervical cancer is the 10th most common cancer...,diseases-and-conditions,"b'<div class=""ExternalClass4A8C8F03738343228D5...",Health Promotion Board,False,True,"[HPV Vaccination FAQ, Getting Support (for STI...",None,"[[HPV Vaccination FAQ, https://www.healthhub.s...","[[Link Between Cervical Cancer and HPV, h2], [...","[Mother and Daughter, Diagram of Cervical Canc...",The best protection against cervical cancer is...,9,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519,1445673,Healthy Food for Kids and Teens,Healthy Food for Kids and Teens,"Food and Nutrition,",https://www.healthhub.sg/live-healthy/A Health...,Good nutrition for kids plays an important rol...,live-healthy-articles,"b'<div class=""ExternalClass7D45F77CD3864AB0BEE...",Health Promotion Board,True,True,[Guilt-Free Treats to Meet Your Child's Nutrit...,"[[[Food Groups, Recommended number of servings...","[[My Healthy Plate, http://www.healthhub.sg/pr...",[[Meeting nutritional needs of kids and teenag...,[],Meeting nutritional needs of kids and teenager...,7,normal
1520,1445358,How to Study Difficult Subjects,How to Study Difficult Subjects,"Mind and Balance,",https://www.healthhub.sg/live-healthy/how_to_s...,Your brain is a muscle. You can train it too!,live-healthy-articles,"b'<div class=""ExternalClass7A329CBEFD0645DE957...",Health Promotion Board,False,True,[],None,"

In [19]:
print(df_hpb.groupby(["reading_level"])["id"].count().sort_values(ascending=False))

reading_level
normal       602
hard          60
very hard      4
Name: id, dtype: int64
